## Исследование сервиса чтения книг по подписке

Цель исследования: первичный анализ базы данных сервиса. Полученные данные помогут сформулировать ценностное предложение для нового продукта.


### Содержание

[1. Подсоединение к базе данных и создание функции вывода результатов SQL-запроса](#chapter1)

[2. Изучение таблиц](#chapter2)

[3. Запросы:](#chapter3)
    
[4. Выводы по запросам](#chapter4)

### Подсоединение к базе данных и создание функции вывода результатов SQL-запроса <a id="chapter1"></a>

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# Функция вывода sql-запроса
def my_query(query):
    
    """
    Эта функция выводит sql-запрос
    
    Ее аргумент - текст запроса
    
    """
    df = pd.io.sql.read_sql(query, con = engine)
    display(df)

### Изучение таблиц <a id="chapter2"></a>

In [3]:
table1 = '''
SELECT * FROM books
LIMIT 5
'''

my_query(table1)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
table2 = '''
SELECT * FROM authors
LIMIT 5
'''

my_query(table2)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
table3 = '''
SELECT * FROM ratings
LIMIT 5
'''

my_query(table3)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
table4 = '''
SELECT * FROM reviews
LIMIT 5
'''

my_query(table4)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [7]:
table5 = '''
SELECT * 
FROM publishers
LIMIT 5
'''

my_query(table5)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Запросы <a id="chapter3"></a>

In [8]:
# Посчитаем, сколько книг вышло после 1 января 2000 года

query1 = '''

SELECT COUNT(book_id) AS num_books_since_2000
FROM books
WHERE publication_date >= '2000-01-01'

'''

my_query(query1)

,num_books_since_2000
0,821


In [9]:
# Для каждой книги посчитаем количество обзоров и среднюю оценку

query2 = '''

WITH books_reviews AS
(
SELECT books.title AS title, books.book_id AS book_id, COUNT(reviews.review_id) AS num_reviews
FROM books
INNER JOIN reviews
ON reviews.book_id = books.book_id
GROUP BY books.title, books.book_id
)

SELECT books_reviews.title, books_reviews.num_reviews, ROUND (AVG(ratings.rating), 2) AS avg_rating
FROM books_reviews
INNER JOIN ratings
ON ratings.book_id = books_reviews.book_id
GROUP BY books_reviews.title, books_reviews.num_reviews
ORDER BY books_reviews.num_reviews DESC
LIMIT 25
'''

my_query(query2)

,title,num_reviews,avg_rating
0,Twilight (Twilight #1),7,3.66
1,The Catcher in the Rye,6,3.83
2,The Book Thief,6,4.26
3,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
4,The Alchemist,6,3.79
5,Outlander (Outlander #1),6,4.13
6,The Hobbit or There and Back Again,6,4.13
7,The Road,6,3.77
8,Water for Elephants,6,3.98
9,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41


In [10]:
# Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

query3 = '''
SELECT publishers.publisher, COUNT(books.book_id) AS num_books
FROM publishers
JOIN books
ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY num_books DESC
LIMIT 1
'''

my_query(query3)

,publisher,num_books
0,Penguin Books,42


In [11]:
# Определим автора с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками)

query4 = '''
WITH avg_ratings AS (

SELECT book_id, AVG(rating) AS book_avg_rating
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) >= 50

) 

SELECT authors.author, ROUND (AVG (avg_ratings.book_avg_rating), 3) AS author_avg_rating
FROM authors
JOIN books
ON books.author_id = authors.author_id
JOIN avg_ratings
ON avg_ratings.book_id = books.book_id
GROUP BY authors.author
ORDER BY author_avg_rating DESC

'''

my_query(query4)

,author,author_avg_rating
0,J.K. Rowling/Mary GrandPré,4.284
1,Markus Zusak/Cao Xuân Việt Khương,4.264
2,J.R.R. Tolkien,4.258
3,Louisa May Alcott,4.192
4,Rick Riordan,4.081
5,William Golding,3.901
6,J.D. Salinger,3.826
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.788
9,Dan Brown,3.755


In [12]:
# Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

query5 = '''
WITH ratings_per_user AS (
SELECT username, COUNT(rating_id) AS num_ratings
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 50
),

reviews_and_ratings_per_user AS
(
SELECT ratings_per_user.username, COUNT(reviews.review_id) AS num_reviews
FROM ratings_per_user
JOIN reviews
ON ratings_per_user.username = reviews.username
GROUP BY ratings_per_user.username
)
SELECT ROUND (AVG (num_reviews), 2) AS avg_num_reviews
FROM reviews_and_ratings_per_user
'''

my_query(query5)

,avg_num_reviews
0,24.33


### Выводы по запросам <a id="chapter4"></a>

1. С 1 января 2000 года вышла 821 книга
2. Больше всего (7) обзоров было про книгу Twilight ("Сумерки"). Это роман о любви девушки-подростка и вампира. В числе наиболее читаемых книг (5-7 обзоров) - классика (например, Шекспир), остросюжетные романы и книги в жанре волшебной сказки (или фэнтези) , такие как "Гарри Поттер" и "Властелин колец". В целом, в топе много книг, интересных  для тинейджеров (не только фэнтези, но и "Над пропастью во ржи", например).
3. Издательство, выпустившее больше других относительно толстых книг - Penguin Books.
4. Самая высокая средняя оценка книг у Джоан Роулинг, автора "Гарри Поттера".
5. Среднее число обзоров для книг, у которых более 50 оценок - 24, то есть на каждый обзор приходится в среднем более двух оценок.